In [1]:
import pandas as pd
import numpy as np

from context import community_module
from community_module.community_detection.similarityCommunityDetection import SimilarityCommunityDetection
from community_module.community_detection.explainedCommunitiesDetection import ExplainedCommunitiesDetection
from community_module.visualization.gephiVisualization import GephiVisualization

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
interactions_df = pd.read_csv('../../data/IMMA/citizenInteractions.csv')
interactions_df.head()

,userResponseID,artworkID,numInteractions
0,81PfE8l9tbFhtvxOE6X5VOh2UpBiYgCO,60c0927e0ee8234e932e7a1d,1
1,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,60c08c580ee8234e932e7a18,1
2,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,60c0908cb5b26479a17d1b33,1
3,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,60c09225b5b26479a17d1b3a,1
4,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,60c0927e0ee8234e932e7a1d,1


In [3]:
users_interactions = pd.pivot_table(interactions_df, values='numInteractions', index='userResponseID', columns='artworkID', fill_value=0)


In [4]:
community_dectection = ExplainedCommunitiesDetection(users_interactions, SimilarityCommunityDetection, 'cosine')

In [21]:
n_communities, users_communities = community_dectection.search_all_communities(percentage=0.78)

In [6]:
users_interactions['community'] = users_communities.values()
users_interactions.reset_index(inplace=True)

In [7]:
# Pintamos en Gephi
gv = GephiVisualization(workspace='imma')

# Preparamos los datos de usuarios y distancias
users = users_interactions[['userResponseID','community']].values
data = users_interactions.iloc[:,1:-1].values
distances = cosine_similarity(data)

gv.load_community(users, distances, users_properties=['community'])

In [23]:
users_interactions[['userResponseID', 'community']]

artworkID,userResponseID,community
0,81PfE8l9tbFhtvxOE6X5VOh2UpBiYgCO,0
1,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,2
2,KL32ozS4irwGu3JtWd0x5p8AV9kXlS9I,6
3,P4M8Uto1UwzTd3OKOns3LZ7hIxBheAg1,0
4,SUw10XRrB4A03Ral1MKas4sLgIW9P3lh,3
5,ScOlinhmp6Rxo9aqywrKMR8J0S90qNQA,2
6,ZqNE3fNu8JXVIadgc5epXYofaOtgEE7F,5
7,aQIxJWU4S2YAOGA93ItHtmuEeNbhudnY,2
8,iCY4NVfyljZxazkltmZh5YiAnY2b3Rm4,1
9,k5SXzdwN9UPY2FXUC1vavDtk3CykTKXR,1


In [31]:
users_without_community = []
for c in range(n_communities):
    community_data = community_dectection.get_community(c, answer_binary=True)

    if len(community_data['members']) > 1:

        print('---------------------')
        print('COMMUNITY -', community_data['name'])
        print('\t- N. Members:', len(community_data['members']))
        print('\t- Properties:')

        for k in community_data['properties'].keys():
            print('\t\t-', k)
    else:
        users_without_community.extend(community_data['members'])

print('---------------------')
print('N. USERS WITHOUT COMMUNITY -', len(users_without_community))

---------------------
COMMUNITY - 0
	- N. Members: 2
	- Properties:
		- 60c0927e0ee8234e932e7a1d
---------------------
COMMUNITY - 1
	- N. Members: 2
	- Properties:
		- 60c09437b5b26479a17d1b3c
		- 60c094dcb78b241f4625bbb9
---------------------
COMMUNITY - 2
	- N. Members: 3
	- Properties:
		- 60c08c580ee8234e932e7a18
		- 60c0908cb5b26479a17d1b33
		- 60c09225b5b26479a17d1b3a
		- 60c0927e0ee8234e932e7a1d
		- 60c09476b84c4b312635d34c
		- 60c095321262e65eec197529
---------------------
N. USERS WITHOUT COMMUNITY - 4
